<a href="https://colab.research.google.com/github/Ayushverma135/Whisper-Hindi-ASR-model-IIT-Bombay-Intership/blob/main/Final_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whisper installation

In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-011ls28s
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-011ls28s
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-

# Load the whisper Model

In [ ]:
import whisper

model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:31<00:00, 99.5MiB/s]


# Mount the Google drive and exracting the data tar file


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tarfile

tar_ref = tarfile.open('/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h.tar.gz', 'r')
tar_ref.extractall('/content/drive/MyDrive/Colab Notebooks/ASR/')
tar_ref.close()

# Displaying the audio from the google drive

In [ ]:
from IPython.display import Audio, display

display(Audio('/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/Audio/01-00031-03.mp3', autoplay=True))

# Transcribing a sample mp3 file from the data


In [ ]:
# original text
# 01-00031-03 क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल मीडिया के माध्यमों पर अपराध करने वालों को पकड़ने में ही असमर्थ है

# transcribed text
result = model.transcribe('/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/Audio/01-00031-03.mp3',language='hi')
print(result["text"])

 क्योंकि हमारी पुलिस का फेस्बुक और गुशे सुसल मिग्रे के माध्यमों पर अप्राज करने वालों को पकड़ने में ही असमर्थ है।


# Transcribe the Audio Files into Text and Write

In [6]:
directory = '/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/Audio/'
file_path = '/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/mp3txt.txt'  #Text file containg names of audio files
files = []

with open(file_path, "r") as file:
    for line in file:
        # Split each line by whitespace to separate the initial numbers representing name of mp3 file and the file path
        parts = line.strip().split("\t")
        if len(parts) > 0:
            # Get the initial numbers (the first part) representing name of mp3 audio file
            number = parts[0]
            mp3file_path = directory + number + ".mp3"
            # Adding path of mp3 files into the list
            files.append(mp3file_path)

In [5]:
len(files),type(files)

(1032, list)

In [7]:
files[0:9]

['/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/Audio/02-17489-01.mp3',
 '/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/Audio/02-13513-01.mp3',
 '/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/Audio/01-01255-01.mp3',
 '/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/Audio/01-06364-01.mp3',
 '/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/Audio/01-01003-01.mp3',
 '/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/Audio/02-20297-01.mp3',
 '/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/Audio/01-09395-01.mp3',
 '/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/Audio/02-18119-01.mp3',
 '/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/Audio/01-08414-01.mp3']

In [ ]:
# Opening the transcribed_audio.txt file to write results
with open('/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/transcribed/transcribed_audio.txt', 'w') as file:
    # Iterate over each mp3 audio file
    for mp3file in files:
        result = model.transcribe(mp3file, language='hi')
        filename = mp3file.split('/')[-1].split('.')[0]
        file.write(filename + ': ' + result["text"] + '\n')

# Word Error Rate(WER)

In [87]:
def wer(reference, hypothesis):
    """
    Calculate Word Error Rate (WER) between reference and hypothesis texts.
    Args:
        reference (str): Reference text (ground truth).
        hypothesis (str): Hypothesis text (transcribed).
    Returns:
        float: Word Error Rate (WER).
    """
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    # Create a matrix to store edit distances between all pairs of reference and hypothesis words
    dp = [[0] * (len(hypothesis_words) + 1) for _ in range(len(reference_words) + 1)]

    # Initialize first row and first column of the matrix
    reference_len=len(reference_words) + 1
    hypothesis_len=len(hypothesis_words) + 1
    for i in range(reference_len):
        dp[i][0] = i
    for j in range(hypothesis_len):
        dp[0][j] = j

    # Fill the matrix
    for i in range(1, reference_len):
        for j in range(1, hypothesis_len):
            if reference_words[i - 1] == hypothesis_words[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j - 1], dp[i - 1][j], dp[i][j - 1]) + 1

    # Returning the WER
    return dp[len(reference_words)][len(hypothesis_words)] / len(reference_words)

## Comparing Transcribed File with Kathbath Dataset Reference File

In [85]:
text="01-00008-03: युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या पैसे खर्च करने की ज़रुरत नहीं है  दिन"
# Define the pattern to extract Hindi sentences
pattern = r'[\u0900-\u097F\s]+'

# Find all matches of Hindi sentences
hindi_sentences = re.findall(pattern, text)

# Join the matches to form the complete Hindi sentence
complete_hindi_sentence = ''.join(hindi_sentences)

print(complete_hindi_sentence)
hindi_sentences

 युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या पैसे खर्च करने की ज़रुरत नहीं है  दिन


[' युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या पैसे खर्च करने की ज़रुरत नहीं है  दिन']

In [7]:
text = "01-00008-03: युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या पैसे खर्च करने की ज़रुरत नहीं है  दिन"

# Split the text based on the colon (":") character
parts = text.split(":")
if len(parts) >= 1:
    number = parts[0].strip()
    print("Extracted number:", number)
else:
    print("Invalid format: Unable to extract number")


Extracted number: 01-00008-03


In [86]:
import re
def wer_from_files(reference_file, hypothesis_file):
    """
    Calculate Word Error Rate (WER) between texts from reference and hypothesis(transcribed) text files.
    Args:
        reference_file (str): File path to the reference text file (ground truth).
        hypothesis_file (str): File path to the hypothesis text file (transcribed).
    Returns:
        float: Word Error Rate (WER).
    """
    # Read reference text from file
    with open(reference_file, 'r', encoding='utf-8') as f:
        reference_text = f.read()
        pattern = r'[\u0900-\u097F\s]+'

        # Find all matches of Hindi sentences
        hindi_sentences = re.findall(pattern, reference_text)

    # Read hypothesis text from file
    with open(hypothesis_file, 'r', encoding='utf-8') as f:
        hypothesis_text = f.read().split(':')[-1]

    # Calculate total number of words in reference text
    total_words = sum(len(line.split()) for line in hindi_sentences)

    # Calculate total number of errors between reference and hypothesis texts
    total_errors = sum(wer(reference_line, hypothesis_line) * len(reference_line.split()) for reference_line, hypothesis_line in zip(hindi_sentences, hypothesis_text))

    wer_score = total_errors / total_words
    return wer_score

In [88]:
# Example usage with file paths:
reference_file_path = '/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/transcribed/text.txt'
hypothesis_file_path = '/content/drive/MyDrive/Colab Notebooks/ASR/GV_Eval_3h/transcribed/transcribed_audio.txt'
wer_score = wer_from_files(reference_file_path, hypothesis_file_path)
print(f"WER: {wer_score * 100:.2f} %")

WER: 15.96 %


In [90]:
import sys
print(sys.version)


3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
